# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

print(os.getcwd())

C:\Users\jeffk\OneDrive\Springboard\GuidedCapstone\notebooks


**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [2]:
df1 = pd.read_csv('../data/step3_output.csv')
df1.drop(df1.columns[[0,1]],axis=1, inplace=True)
df1.head(5)

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,kmeans
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,1
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,1
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,0
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [3]:
states = pd.get_dummies(df1.state)

df = pd.concat([df1, states], axis=1)
del(df['state'])

Create dummy variables for the kmeans categories, and remove the original column for kmeans too:

In [4]:
kmeans_dummies = pd.get_dummies(df1.kmeans, prefix='kmeans')
df = pd.concat([df, kmeans_dummies], axis=1)
del(df['kmeans'])

## Select features for model input and output

**<font color='teal'> You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [5]:
outputColumns = ['AdultWeekend','AdultWeekday','projectedDaysOpen', 'daysOpenLastYear'] #
holdOutColumns = ['kmeans_0','kmeans_1','kmeans_2']
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

Input = df.drop(['Name'] + outputColumns + holdOutColumns,axis=1)
Output = df[outputColumns]

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selectionm import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Input, Output, test_size=0.25, random_state=25)

Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score,mean_absolute_error

steps = [
    ('scale', StandardScaler()),
    ('estimator', LinearRegression())
]
model1 = Pipeline(steps)
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
model_1_scores = (explained_variance_score(y_test,y_pred,multioutput='raw_values')[0],
                  mean_absolute_error(y_test,y_pred,multioutput='raw_values')[0])

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [8]:
print('Explained variance:', explained_variance_score(y_test,y_pred,multioutput='raw_values')[0])
print('Mean absolute error:', mean_absolute_error(y_test,y_pred,multioutput='raw_values')[0])

Explained variance: -0.14101586063132543
Mean absolute error: 12.862671425450909


## Review Model Outcomes — Iterate over additional models as needed.

**<font color='teal'> Print the intercept value from the linear model. </font>**

In [9]:
print('Intercepts:', model1.named_steps['estimator'].intercept_)

Intercepts: [ 64.7870849   58.75851779 120.35504663 115.01393484]


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [10]:
coefficients = dict(zip(Input.columns.tolist(),model1.named_steps['estimator'].coef_[0]))
for key, value in sorted(coefficients.items(),key=lambda item: abs(item[1]), reverse = True):
    print ('%s : %s' % (key, coefficients[key]))

New York : -48012287610947.74
Michigan : -43424400741852.91
Colorado : -40299001548002.586
California : -36808480178562.375
Vermont : -35545572410632.95
Montana : -34223975916521.31
New Hampshire : -34223975916512.92
Pennsylvania : -34223975916511.945
Utah : -32836605209116.637
Wisconsin : -32836605209111.59
Idaho : -31374736020798.016
Minnesota : -29827416644426.74
Massachusetts : -29827416644424.61
Oregon : -28180575075334.53
Washington : -28180575075334.418
Maine : -28180575075330.6
Wyoming : -22418089187248.285
New Mexico : -20092734235665.957
North Carolina : -20092734235662.46
Connecticut : -20092734235661.266
Virginia : -20092734235660.996
Alaska : -17436585672463.738
Ohio : -17436585672463.084
Nevada : -17436585672461.832
West Virginia : -17436585672461.6
South Dakota : -14266056755520.793
Illinois : -14266056755519.344
Iowa : -14266056755519.295
Indiana : -14266056755518.893
Missouri : -14266056755518.836
Arizona : -10108191499009.533
Tennessee : -10108191499008.291
Maryland :

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

## Predict target for Big Mountain Resort

In [11]:
#Big Mountain prediction
whitefish_index = df[df.Name == 'Whitefish Mountain Resort'].index.tolist()
BigMountain_features = Input.iloc[whitefish_index]
BigMountain_target = Output.iloc[whitefish_index]

model1_output = model1.predict(BigMountain_features)

print('Target and model predictions:')

for i in range(1):
    print(outputColumns[i],BigMountain_target.iat[0,i], model1_output[0][i])

Target and model predictions:
AdultWeekend 81.0 78.27145990422143


## Rebuild model without states as features

In [12]:
Input = df.drop(['Name'] + outputColumns + holdOutColumns + list(states.columns),axis=1)
Output = df[outputColumns]

X_train, X_test, y_train, y_test = train_test_split(Input, Output, test_size=0.25, random_state=25)

#### Model 2

In [13]:
steps = [
    ('scale', StandardScaler()),
    ('estimator', LinearRegression())
]
model2 = Pipeline(steps)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

print('Interecpt:' , model2.named_steps['estimator'].intercept_, '\n')
coefficients = dict(zip(X_test.columns.tolist(), model2.named_steps['estimator'].coef_[0]))
for key, value in sorted(coefficients.items(),key=lambda item: abs(item[1]), reverse = True):
    print ('%s : %s' % (key, coefficients[key]))

model_2_scores = (explained_variance_score(y_test,y_pred,multioutput='raw_values'),mean_absolute_error(y_test,y_pred,multioutput='raw_values')[0])

Interecpt: [ 64.78497482  58.75614681 120.35228394 115.01194259] 

fastQuads : 9.517971938680525
SkiableTerrain_ac : -7.029828413813089
vertical_drop : 6.811453393251539
Runs : 5.999660980756762
quad : 4.2415126331871935
surface : -3.995889036564575
summit_elev : -3.8996185107099146
base_elev : 3.1580675320397438
fastEight : -2.689795122089791
total_chairs : 2.483202515964191
trams : -2.3757363779418412
triple : 2.024769059565143
double : 1.6830036521171388
averageSnowfall : 1.6079740907341744
fastSixes : 1.273546651427947
LongestRun_mi : 1.2058243678006395
Snow Making_ac : -1.1604993473157963
NightSkiing_ac : 0.7655607239933941
yearsOpen : -0.5114449613632013
TerrainParks : -0.06891459581766324


**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

In [14]:
#Big Mountain prediction
whitefish_index = df[df.Name == 'Whitefish Mountain Resort'].index.tolist()
BigMountain_features = Input.iloc[whitefish_index]
BigMountain_target = Output.iloc[whitefish_index]

model2_output = model2.predict(BigMountain_features)

print('Target and model predictions:')

for i in range(1):
    print(outputColumns[i],BigMountain_target.iat[0,i], model2_output[0][i])

Target and model predictions:
AdultWeekend 81.0 89.55711002724041


#### Model 3

In [15]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

Input = df.drop(['Name', 'summit_elev','base_elev'] + outputColumns + holdOutColumns + list(states.columns),axis=1)
Output = df[outputColumns]

X_train, X_test, y_train, y_test = train_test_split(Input, Output, test_size=0.25, random_state=25)

steps = [
    ('scale', StandardScaler()),
    ('estimator', LinearRegression())
]
model3 = Pipeline(steps)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)

print('Interecpt:' , model3.named_steps['estimator'].intercept_, '\n')
coefficients = dict(zip(X_test.columns.tolist(), model3.named_steps['estimator'].coef_[0]))
for key, value in sorted(coefficients.items(),key=lambda item: abs(item[1]), reverse = True):
    print ('%s : %s' % (key, coefficients[key]))

model_3_scores = (explained_variance_score(y_test,y_pred,multioutput='raw_values'),mean_absolute_error(y_test,y_pred,multioutput='raw_values')[0])

Interecpt: [ 64.78497482  58.75614681 120.35228394 115.01194259] 

fastQuads : 9.54598716225511
SkiableTerrain_ac : -6.999846104542714
Runs : 5.875464636402548
vertical_drop : 5.8059504928367955
quad : 4.261084055336437
surface : -3.9938456871508277
fastEight : -2.6320360423210643
total_chairs : 2.485090970678938
trams : -2.3839846769137063
triple : 2.0345120670185244
double : 1.6922794940467314
averageSnowfall : 1.5590821629389287
fastSixes : 1.2863384862771543
LongestRun_mi : 1.2030652141837908
Snow Making_ac : -1.1448588251607152
NightSkiing_ac : 0.7491056540364434
yearsOpen : -0.4939674604864326
TerrainParks : -0.04919959501044707


In [16]:
#Big Mountain prediction

BigMountain_features = Input.iloc[whitefish_index]
BigMountain_target = Output.iloc[whitefish_index]

model3_output = model3.predict(BigMountain_features)

print('Target and model predictions:')

for i in range(1):
    print(outputColumns[i],BigMountain_target.iat[0,i], model3_output[0][i])

Target and model predictions:
AdultWeekend 81.0 89.52036895173944


In [17]:
print('Model 1:', model_1_scores)
print('Model 2:', model_2_scores)
print('Model 3:', model_3_scores)

Model 1: (-0.14101586063132543, 12.862671425450909)
Model 2: (array([-0.98223029, -0.45997456, -0.48061408, -0.65739792]), 14.012379882004051)
Model 3: (array([-0.96073626, -0.35003002, -0.20552946, -0.66815738]), 13.955354904778455)


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -0.397 | 12.86 | |
| Model 2. | -0.645 | 14.01 |'state'|
| Model 3. | -0.546 | 13.95 |'state','summit_elev','base_elev'|

# Model 1 is selected for having lowest mean absolute error.

In [18]:
y_true_1D = [1, 2, 3, 4]
y_pred_1D = [2, 3, 4, 5]
mean_absolute_error(y_true_1D,y_pred_1D)

1.0

In [19]:
y_true_2D = [[1,2], [2,2], [3,2], [4,2]]
y_pred_2D = [[2,12], [3,3], [4,3], [5,3]]
mean_absolute_error(y_true_2D, y_pred_2D, multioutput='raw_values')

array([1.  , 3.25])

In [20]:
scale = StandardScaler().fit(Input)
scale.transform(Input)[131,:]
#Input.iloc[131,:]

array([-1.09932085, -0.30893969, -0.05513178, -0.2840776 , -0.46387252,
        0.81409101, -0.30927686, -1.01161941,  1.1567092 ,  0.25282839,
       -0.74361256, -0.65746141, -0.37816525, -0.39044416, -0.57796154,
       -0.02435003, -0.9781918 , -0.23457102])